### Load data

In [28]:
import pandas as pd
books_data = pd.read_csv('data/books_data.csv')
books_ratings = pd.read_csv('data/books_rating.csv')

In [29]:
# Titles
books_data_1 = books_data.drop(['image', 'previewLink', 'infoLink', 'publisher', 'ratingsCount'], axis=1, inplace=False)
books_ratings_1 = books_ratings.drop(['Id','Price', 'profileName', 'review/time'], axis=1, inplace=False)

books_data_1.rename(columns={'Title': 'title','publishedDate': 'published_date'}, inplace=True)
books_ratings_1.rename(columns={'Title': 'title','User_id':'user_id',  'review/score': 'score','review/helpfulness': 'helpfulness', 'review/text': 'text', 'review/summary': 'summary'}, inplace=True)

print(list(books_data_1.columns))
print(list(books_ratings_1.columns))

['title', 'description', 'authors', 'published_date', 'categories']
['title', 'user_id', 'helpfulness', 'score', 'summary', 'text']


In [40]:
books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: s.lower() if type(s) == str else s)
books_ratings_1['title'] = books_ratings_1['title'].replace(r"\(.*\)","", regex=True)


books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: 'the hobbit' if 'hobbit' in s else s)
books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: 'lord of the rings' if 'lord of the rings' in s else s)
books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: 'pride and prejudice' if 'pride & prejudice' in s else s)
books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: 'pride and prejudice' if 'pride and prejudice' in s else s)
books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: 'the great gatsby' if 'great gatsby' in s else s)
books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: 'to kill a mocking bird' if 'mocking bird' in s else s)
books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: 'to kill a mocking bird' if 'mockingbird' in s else s)

In [31]:
# load book data for books in book ratings
ratings_titles = books_ratings_1['title'].unique()
books_data_1 = books_data_1[books_data_1['title'].isin(ratings_titles)]

In [32]:
n_ratings = books_ratings_1.groupby('title').count()['score']>=10
famous_books = n_ratings[n_ratings].index
books_ratings_1 = books_ratings_1[books_ratings_1['title'].isin(famous_books)]

n_ratings_1 = books_ratings_1['title'].value_counts().rename("n_ratings")

books_data_1 = books_data_1.merge(n_ratings_1, on='title', how='inner')
#print(books_data_1.columns)

In [33]:
x = books_ratings_1.groupby('user_id').count()['score'] > 20
considerable_users = x[x].index
books_ratings_1 = books_ratings_1[books_ratings_1['user_id'].isin(considerable_users)]
#
# print(books_ratings_1.sample(5))

books_ratings_1['user_id'].value_counts()

user_id
A1D2C0WDCSHUWZ    2995
AFVQZQ8PW0L       2398
A14OJS0VWMOSWO    2390
A1X8VZWTOG8IS6    1424
A1K1JW1C5CUSUZ    1218
                  ... 
A2C8S29IIXWXSA      21
A1OIIMUEDFH2VM      21
A2B8XZ9NZKWZK7      21
AEKY9OVY8MOPH       21
A1H11BV01L91I6      21
Name: count, Length: 7721, dtype: int64

In [34]:
# extract all books rated by A14OJS0VWMOSWO
books_ratings_1[books_ratings_1['user_id'] == 'A14OJS0VWMOSWO']['title'].value_counts()

title
alice's adventures in wonderland                                      12
great expectations                                                    10
the white company                                                      9
jane eyre                                                              7
the billboard book of top 40 r and b and hip-hop hits                  7
                                                                      ..
mole & the baby bird                                                   1
the interior designers guide to pricing estimating and budgeting       1
the reluctant dragon                                                   1
phonics pathways: clear steps to easy reading and perfect spelling     1
he walked among us                                                     1
Name: count, Length: 2041, dtype: int64

In [35]:
avg_rating = books_ratings_1.groupby('title')["score"].mean().rename("avg_rating")
books_data_1 = books_data_1.merge(avg_rating, on='title', how='inner')
#print(books_data_1.columns)


In [36]:
def get_n_helpful(x):
    if isinstance(x, str) and '/' in x:
        num, denom = x.split('/')
        return int(denom)
    else:
        return x

def replace_fraction(x):
    if isinstance(x, str) and '/' in x:
        num, denom = x.split('/')
        if denom == '0':
            return 0
        else:
            return int(num) / int(denom)
    else:
        return x

books_ratings_1['helpfulness_count'] = books_ratings_1['helpfulness'].apply(get_n_helpful)
books_ratings_1['helpfulness_pct'] = books_ratings_1['helpfulness'].apply(replace_fraction)

#print(books_ratings_1.sample(2))

In [37]:
#print(books_data_1.columns)

In [38]:
#def extract_year(x):
#    year1 = pd.to_datetime(x, format='mixed').year
##    print(year1)
#    return year1
    
#books_data_1['published_date'] = books_data_1['published_date'].apply(extract_year)
#p#rint(books_data_1.sample(5))


In [39]:
import os
if not os.path.exists('clean_data'):
    os.makedirs('clean_data')

# save clean data
books_data_1.to_csv('clean_data/books_data_clean.csv')
books_ratings_1.to_csv('clean_data/books_rating_clean.csv')

print("First we had {} books and {} ratings".format(len(books_data), len(books_ratings)))
print("Now we have {} books and {} ratings".format(len(books_data_1), len(books_ratings_1)))

First we had 212404 books and 3000000 ratings
Now we have 75 books and 431293 ratings
